In [ ]:
!pip install pypdf
!pip install langchain_openai
!pip install -U langchain-community
!pip install chromadb

In [ ]:
import os
import urllib.request
import gradio as gr

from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("API_KEY_OPENAI")

# 데이터 로드 및 전처리

In [ ]:
urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch07/2020_%EA%B2%BD%EC%A0%9C%EA%B8%88%EC%9C%B5%EC%9A%A9%EC%96%B4%20700%EC%84%A0_%EA%B2%8C%EC%8B%9C.pdf",
                           filename = '2020_경제금융용어 700선_게시.pdf')

In [ ]:
loader = PyPDFLoader('2020_경제금융용어 700선_게시.pdf')
texts = loader.load_and_split()
print('문서의 수 :', len(texts))

In [ ]:
print(texts[0].page_content)

In [ ]:
print(texts[13].page_content)

In [ ]:
texts = texts[13:]
print('줄어든 청크의 수 :', len(texts))

In [ ]:
print(texts[-1].page_content)

In [ ]:
texts = texts[:-1]
print('줄어든 청크의 수 :', len(texts))

# 임베딩 및 벡터 데이터베이스 적재

In [ ]:
embedding = OpenAIEmbeddings(chunk_size = 100)

vectordb = Chroma.from_documents(documents = texts, embedding = embedding)
print(vectordb._collection.count())

In [ ]:
for key in vectordb._collection.get():
  print(key)

In [ ]:
document = vectordb._collection.get()['documents']
print('청크의 개수 :', len(document))
print('-' * 100)
print("0번 청크 출력 :", document[0])

In [ ]:
embedding = vectordb._collection.get(include = ['embeddings'])['embeddings']
print('임베딩 벡터의 개수 :', len(embedding))

In [ ]:
print('0번 청크의 임베딩 값 :', embedding[0])
print('-' * 100)
print('0번 청크의 임베딩 값의 길이 :', len(embedding[0]))

In [ ]:
retriever = vectordb.as_retriever(search_kwargs = {"k" : 2})

docs = retriever.get_relevant_documents("비트코인이 궁금해")
print("유사 문서 개수 :", len(docs))
print('-' * 100)

print('첫 번째 유사 문서 :', docs[0].page_content)
print('-' * 100)

print('두 번째 유사 문서 :', docs[1].page_content)

# 프롬프트 템플릿, LLM, 리트리버의 연결

In [ ]:
template = '''당신은 한국은행에서 만든 금융 용어를 설명해주는 금융쟁이입니다. 주어진 검색 결과를 바탕으로 답변하세요.
검색 결과에 없는 내용이라면 답변할 수 없다고 하세요. 반말로 친근하게 답변하세요.
{context}

Question : {question}

Answer :
'''

prompt = PromptTemplate.from_template(template)

In [ ]:
llm = ChatOpenAI(model_name = 'gpt-4o', temperature = 0)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type_kwargs = {"prompt" : prompt},
    retriever = retriever,
    return_source_documents = True
)

In [ ]:
input = "디커플링은 무엇인가?"
response = qa_chain(input)
print(response)

In [ ]:
def get_response(input_text):
  res = qa_chain(input_text)
  return res['result'].strip()

In [ ]:
input_text = "너의 역할은 무엇이니??"
result = get_response(input_text)
print(result)

In [ ]:
input_text = "비트코인에 대해서 궁금해"

result = get_response(input_text)
print(result)

In [ ]:
# 인터페이스 생성
with gr.Blocks() as demo:
  chatbot = gr.Chatbot(label = '경제 금융 용어 챗봇')   # 챗봇 레이블을 좌측 상단에 구성
  msg = gr.Textbox(label = "질문해주세요!")   # 하단의 채팅창 레이블
  clear = gr.Button("대화 초기화")   # 대화 초기화 버튼

  # 챗봇의 답변을 처리하는 함수
  def respond(message, chat_history):
     result = qa_chain(message)
     bot_message = result['result']

     # 채팅 기록에 사용자의 메시지의 봇의 응답을 추가
     chat_history.append((message, bot_message))
     return "", chat_history

  # 사용자의 입력을 제출 하면 respond 함수가 호출
  msg.submit(respond, [msg, chatbot], [msg, chatbot])

  # '초기화' 버튼을 클릭하면 채팅 기록을 초기화
  clear.click(lambda : None, None, chatbot, queue = False)

demo.launch(debug = True)